##### CS 583 Assignment 2 
##### by Daniyal Mufti
##### Character Level Neural Machine Translation - English to Pig Latin 
##### Model 2 - Using a Sequence to Sequence Bidirectional LSTM Encoder Decoder Model

### 1. Setup

In [2]:
#Import Libraries and Dependecies
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dropout, Dense, Embedding, Bidirectional, Add, Concatenate, Dropout
from tensorflow.keras import Input, Model

In [3]:
# Set hyperparamters
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "data.txt"

### 2. Vectorize Data

In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split(" ")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = " " + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
#print(input_token_index)
#print(target_token_index)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index["'"]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 6510
Number of unique input tokens: 40
Number of unique output tokens: 42
Max sequence length for inputs: 18
Max sequence length for outputs: 24


### 3. Build the Bidirectional LSTM Encoder Decoder Model

In [5]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = Bidirectional(LSTM(128, return_state=True))
encoder_output, forw_state_h, forw_state_c, back_state_h, back_state_c = encoder(encoder_inputs)
state_h_final = Concatenate()([forw_state_h, back_state_h])
state_c_final = Concatenate()([forw_state_c, back_state_c])

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h_final, state_c_final]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 40)]   0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 256), (None, 173056      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 42)]   0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 256)          0           bidirectional[0][1]              
                                                                 bidirectional[0][3]          

### 4. Compile and Fit Model

In [6]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2sAM2")

Epoch 1/100
82/82 [==============================] - 9s 19ms/step - loss: 1.4809 - accuracy: 0.5987 - val_loss: 1.1777 - val_accuracy: 0.6951
Epoch 2/100
82/82 [==============================] - 1s 11ms/step - loss: 1.1057 - accuracy: 0.6818 - val_loss: 0.9535 - val_accuracy: 0.7332
Epoch 3/100
82/82 [==============================] - 1s 11ms/step - loss: 0.9160 - accuracy: 0.7393 - val_loss: 0.8094 - val_accuracy: 0.7733
Epoch 4/100
82/82 [==============================] - 1s 11ms/step - loss: 0.7544 - accuracy: 0.7775 - val_loss: 0.6727 - val_accuracy: 0.8005
Epoch 5/100
82/82 [==============================] - 1s 11ms/step - loss: 0.6198 - accuracy: 0.8112 - val_loss: 0.5565 - val_accuracy: 0.8283
Epoch 6/100
82/82 [==============================] - 1s 11ms/step - loss: 0.5118 - accuracy: 0.8413 - val_loss: 0.4844 - val_accuracy: 0.8560
Epoch 7/100
82/82 [==============================] - 1s 11ms/step - loss: 0.4248 - accuracy: 0.8701 - val_loss: 0.4025 - val_accuracy: 0.8734
Epoch 

82/82 [==============================] - 1s 11ms/step - loss: 0.0045 - accuracy: 0.9990 - val_loss: 0.0513 - val_accuracy: 0.9865
Epoch 59/100
82/82 [==============================] - 1s 11ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 0.0476 - val_accuracy: 0.9873
Epoch 60/100
82/82 [==============================] - 1s 11ms/step - loss: 0.0044 - accuracy: 0.9990 - val_loss: 0.0428 - val_accuracy: 0.9888
Epoch 61/100
82/82 [==============================] - 1s 11ms/step - loss: 0.0041 - accuracy: 0.9991 - val_loss: 0.0405 - val_accuracy: 0.9893
Epoch 62/100
82/82 [==============================] - 1s 11ms/step - loss: 0.0044 - accuracy: 0.9990 - val_loss: 0.0541 - val_accuracy: 0.9863
Epoch 63/100
82/82 [==============================] - 1s 11ms/step - loss: 0.0039 - accuracy: 0.9993 - val_loss: 0.0433 - val_accuracy: 0.9888
Epoch 64/100
82/82 [==============================] - 1s 11ms/step - loss: 0.0037 - accuracy: 0.9993 - val_loss: 0.0558 - val_accuracy: 0.9856
Epoch 65/100

INFO:tensorflow:Assets written to: s2sAM2\assets


INFO:tensorflow:Assets written to: s2sAM2\assets


### 5. Build Inference Model

In [7]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2sAM2")

encoder_inputs = model.input[0]  # input_1
encoder_output, forw_state_h, forw_state_c, back_state_h, back_state_c = model.layers[1].output 
state_h_final = Concatenate()([forw_state_h, back_state_h])
state_c_final = Concatenate()([forw_state_c, back_state_c])
encoder_states = [state_h_final, state_c_final]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[5]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

### 6. Decode Some Sample Sequences

In [8]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index[" "]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [9]:
for seq_index in range(30):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: apprehensive
Decoded sentence: apprehensiveway

-
Input sentence: desperately
Decoded sentence: esperatelyday

-
Input sentence: grew
Decoded sentence: ewgray

-
Input sentence: niggardly
Decoded sentence: iggardlynay

-
Input sentence: brought
Decoded sentence: oughtbray

-
Input sentence: imagery
Decoded sentence: imageryway

-
Input sentence: quicker
Decoded sentence: ickerquay

-
Input sentence: for
Decoded sentence: orfay

-
Input sentence: emergency
Decoded sentence: emergencyway

-
Input sentence: felt
Decoded sentence: eltfay

-
Input sentence: informing
Decoded sentence: informingway

-
Input sentence: withdrawn
Decoded sentence: ithdrawnway

-
Input sentence: bond-street
Decoded sentence: ondbay-eetstray

-
Input sentence: along
Decoded sentence: alongway

-
Input sentence: drove
Decoded sentence: ovedray

-
Input sentence: observant
Decoded sentence: observantway

-
Input sentence: bluntly
Decoded sentence: untlyblay

-
Input sentence: gratitude
Decoded sen